IMPORTING THE LIBRARIES

In [149]:
import json

SETTING OPEN AI CREDENTIALS

In [1]:
import openai

openai.api_type = ""
openai.api_base = ""
openai.api_version = ""
openai.api_key = ''

In [2]:
from PyPDF2 import PdfReader

In [8]:
import ast

import pandas as pd

READING THE PDF AND THEN EXTRACTING ONLY THE FIRST 5 PAGES OF THE PDF TO GET THE TABLE OF CONTENTS INFORMATION

In [54]:
from langchain.chains.summarize import load_summarize_chain

In [3]:
reader = PdfReader("Task1_Input.pdf")

In [4]:
first_pages = reader.pages[0:5]

In [5]:
raw_text = ''
for i in first_pages:
    content = i.extract_text()
    raw_text += content

In [6]:
len(raw_text)

10615

AFTER CONVERTING THE FIRST 5 PAGES OF THE PDF INTO STRING, THIS STRING IS FED TO THE OPEN AI GPT 3.5 MODEL TO FETCH PAGE NOS OF EACH CHAPTER

In [7]:
result = openai.ChatCompletion.create(
    engine="",
    messages=[
        {
          "role": "system",
          "content": "You are a helpful assistant for getting Chapter page numbers only for Chapters with name Change No for a given text.Get me Chapter Names and its corresponding Page Numbers.Please get me only Chapter Names and Page numbers in JSON format like this {'Change No':'Page No'}.Get me Chapter name and Page no for each and every chapter.",
        },
        {
          "role": "user",
          "content": raw_text
        },
    ],
)

CONVERTING THE RESULT OF THE OPEN AI MODEL INTO A DATAFRAME WITH CHAPTER AND PAGE NOS

In [9]:
dictionary = ast.literal_eval(result['choices'][0]['message']['content'])

In [10]:
page_infos = pd.DataFrame(dictionary.items(),columns=['CHAPTER','PAGE NO'])

In [11]:
page_infos = page_infos.rename(columns={'PAGE NO':'STARTING PAGE NO'})

In [13]:
page_infos['STARTING PAGE NO'] = page_infos['STARTING PAGE NO'].astype(int)

In [14]:
page_infos['ENDING PAGE NO'] = page_infos['STARTING PAGE NO'].shift(-1) -1

EXTRACTING EACH CHAPTER CONTENT FROM THE PDF AND SAVING IN A SEPERATE DATAFRAME

In [16]:
chapters_name = []
chapters_content = []

for i,j,k in zip(list(page_infos['CHAPTER']),list(page_infos['STARTING PAGE NO']),list(page_infos['ENDING PAGE NO'])):
    pdf_file = open('Task1_Input.pdf','rb')
    
    pdf_reader = PdfReader(pdf_file)
    
    chapter_content = ''
    
    if k == 0:
        page = reader.pages[j:int(k)]
    else:
        page = reader.pages[j:]
    
    for page_num in page:
        content = page_num.extract_text()
        chapter_content+=content
    
    chapters_name.append(i)
    chapters_content.append(chapter_content)

In [17]:
chap_content = pd.DataFrame(columns=['CHAPTER','CHAPTER CONTENT'])

In [18]:
chap_content['CHAPTER'] = chapters_name

chap_content['CHAPTER CONTENT'] = chapters_content

In [19]:
chap_content

,CHAPTER,CHAPTER CONTENT
0,Change No. 1,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...
1,Change No. 2,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...
2,Change No. 3,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...
3,Change No. 4,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...
4,Change No. 5,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...
5,Change No. 6,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...
6,Change No. 7,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...
7,Change No. 8,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...
8,Change No. 9,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...
9,Change No. 10,HOMEOWNERS \nFORMS FI LING HO -2021 -OFR21 P...


CREATING A SUMMARY CHAIN USING LANGCHAIN FOR SUMMARIZING EACH CHAPTER USING MAP REDUCE FUNCTIONALITY

In [55]:
summary_chain = load_summarize_chain(llm=llm, chain_type='map_reduce',
#                                      verbose=True # Set verbose=True if you want to see the prompts being used
                                    )

CREATING TEXT SPLITTER TO SPLIT EACH CHAPTER INTO MULTIPLE CHUNKS FOR MODEL TO EASILY READ EACH CHAPTERS CONTENT

In [72]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=10000, chunk_overlap=0)

SPLITTING EACH CHAPTER CONTENT AND THEN SUMMARIZING AND THEN SAVING IN JSON

In [76]:
final_json = {}
for i,j in zip(chap_content['CHAPTER'],chap_content['CHAPTER CONTENT']):
    splits = text_splitter.split_text(j)
    docs = text_splitter.create_documents(splits)
    
    result = summary_chain.run(docs)
    
    final_json[i] = result

In [85]:
final_df = pd.DataFrame(final_json.items(),columns=['CHAPTER','SUMMARY OF CHAPTER'])

In [147]:
final_df.to_excel('CHAPTER WISE SUMMARY.xlsx',index=False)

In [148]:
final_json

{'Change No. 1': 'Revisions are being made to homeowners insurance forms to address various issues such as leasing of mineral rights, coverage limits, and coverage for specific items like virtual currency and riding lawn mowers. The changes aim to clarify coverage, broaden or reduce coverage depending on the specific change, and address emerging risks such as cannabis-related property and liability exposures. Additionally, some endorsements are being withdrawn and new forms are being introduced to adapt to changing needs and advancements in technology. Overall, the revisions aim to provide more flexibility and options for insurers and policyholders while maintaining consistency and clarity.',
 'Change No. 2': 'The document outlines revisions to homeowners insurance policies, including changes to the definition of "business" and coverage limits for personal property. ISO is updating its Homeowners policy forms to make changes to coverage for model or hobby aircraft and watercraft, as we

In [150]:
file_path = "CHAPTER WISE SUMMARY.txt"

In [151]:
with open(file_path, 'w') as file:
    json.dump(final_json, file)